In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 
from vllm import LLM, SamplingParams


/home/yilong/miniconda3/envs/vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 03-05 14:02:47 __init__.py:190] Automatically detected platform cuda.


2025-03-05 14:02:47,225	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
#llm = LLM(model="outputs/Qwen-0.5B-GRPO-Tom/checkpoint-563")
#llm = LLM(model="outputs/Qwen-0.5B-GRPO-Tom-512/checkpoint-773")
#llm = LLM(model="outputs/Qwen-0.5B-GRPO-Tom-512/checkpoint-300")
#llm = LLM(model="outputs/Qwen-0.5B-GRPO-Tom-WS/checkpoint-1268")
#llm = LLM(model="outputs/Qwen-0.5B-GRPO-Tom-WS/checkpoint-700")
#llm = LLM(model="outputs/Qwen-0.5B-GRPO-Tom-FT-WS/checkpoint-400")
#llm = LLM(model="outputs/Qwen-0.5B-GRPO-Tom-KK/checkpoint-724")
llm = LLM(model="outputs/Qwen-0.5B-GRPO-TomC-FT-WS/checkpoint-200")


INFO 03-05 14:02:51 config.py:542] This model supports multiple tasks: {'reward', 'score', 'classify', 'embed', 'generate'}. Defaulting to 'generate'.
INFO 03-05 14:02:51 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='outputs/Qwen-0.5B-GRPO-TomC-FT-WS/checkpoint-200', speculative_config=None, tokenizer='outputs/Qwen-0.5B-GRPO-TomC-FT-WS/checkpoint-200', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_nam

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  7.81it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  7.76it/s]



INFO 03-05 14:02:53 model_runner.py:1115] Loading model weights took 0.9277 GB
INFO 03-05 14:02:53 worker.py:267] Memory profiling takes 0.46 seconds
INFO 03-05 14:02:53 worker.py:267] the current vLLM instance can use total_gpu_memory (23.99GiB) x gpu_memory_utilization (0.90) = 21.59GiB
INFO 03-05 14:02:53 worker.py:267] model weights take 0.93GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 1.44GiB; the rest of the memory reserved for KV Cache is 19.14GiB.
INFO 03-05 14:02:53 executor_base.py:110] # CUDA blocks: 104553, # CPU blocks: 21845
INFO 03-05 14:02:53 executor_base.py:115] Maximum concurrency for 32768 tokens per request: 51.05x
INFO 03-05 14:02:56 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:09<00:00,  3.81it/s]

INFO 03-05 14:03:05 model_runner.py:1562] Graph capturing finished in 9 secs, took 0.00 GiB
INFO 03-05 14:03:05 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 12.36 seconds


In [3]:
sampling_params = SamplingParams(temperature=0.6, max_tokens=512, top_p = 0.95)

# read data

In [4]:
from datasets import load_dataset, Dataset
import re
from tqdm import tqdm
import json

In [5]:
# uncomment middle messages for 1-shot prompting
def get_questions() -> Dataset:
    data = load_dataset('parquet', data_files='data/cleaned_train.parquet')['train']
    test_data = load_dataset('parquet', data_files='data/cleaned_test.parquet')['train']
    return data , test_data


dataset, testset = get_questions()
#testset = get_gsm8k_questions('test')




def extract_xml_answer(text: str) -> str:
    answer_pattern = r'<answer>(.*?)</answer>'
    matches = list(re.finditer(answer_pattern, text, re.DOTALL))
    if not matches:
        return None
        
    final_answer = matches[-1].group(1).strip()
    return final_answer



def normalize_answer(answer: str) -> str:
    """Normalizes the answer text for better comparison.
    Args:
        answer: Raw answer text
    Returns:
        Normalized answer text
    """
    # Convert to lowercase
    normalized = answer.lower()
    # Remove extra whitespace
    normalized = re.sub(r'\s+', ' ', normalized).strip()
    # Remove punctuation that doesn't affect meaning
    normalized = re.sub(r'[.,;:!?]', '', normalized)
    return normalized


In [16]:
def reward_func(response, answer):
    response_ = extract_xml_answer(response)
    norm_response = normalize_answer(response_)
    norm_answer = normalize_answer(answer)
    if norm_response == norm_answer:
        return 1
    elif "in the " in norm_response:
        if norm_response.split('in the ')[-1] == norm_answer:
            return 1
    elif "\"" in norm_response:
        if norm_response.split("\"")[-1].split("\"")[0]== norm_answer:
            return 1
    
    return 0

In [7]:
dataset, testset = get_questions()

In [8]:
i = 0
outputs= llm.chat([testset['prompt'][0], testset['prompt'][1], testset['prompt'][2]], sampling_params)
print(outputs[0].outputs[0].text)
print(testset['answer'][0])

print(outputs[1].outputs[0].text)
print(testset['answer'][1])

print(outputs[2].outputs[0].text)
print(testset['answer'][2])

INFO 03-05 14:03:10 chat_utils.py:332] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


Processed prompts: 100%|██████████| 3/3 [00:00<00:00,  7.37it/s, est. speed input: 2806.63 toks/s, output: 324.40 toks/s]

<think> Reasoning process: Charlotte's presence is interrupted by Nicholas's footsteps, leaving her with a brief moment of distraction. She quickly realizes that she will need to search for the walkie-talkie. </think> <answer> Toolbox </answer>
plastic bin
<think> reasoning process: Paige and Emily are likely searching for the silver letter opener together in the conference room. </think> <answer> conference room </answer>
conference room
<think> reasoning process: Peyton's presence and actions in the consulate meeting room suggest that she is likely the one to search for the silver letter opener. </think> <answer> consulate meeting room </answer>
consulate meeting room


In [9]:
def main_func(llm, testset, n, n_repeat=5):
    # Initialize result storage
    all_results = []
    if type(testset)==pd.core.frame.DataFrame:
        testset.index = range(len(testset))
    # Iterate through test set in chunks of n
    with tqdm(range(0, len(testset['prompt']), n), desc="Processing batches") as pbar:
        for i in pbar:
    #for i in tqdm(range(0, len(testset['prompt']), n), desc="Processing batches"):
            # Extract current batch of prompts and answers
            batch_prompts = [p for p in testset['prompt'][i:i+n]]
            batch_answers = [a for a in testset['answer'][i:i+n]]

            # Get outputs from the LLM for the current batch
            for k in range(n_repeat):
                try:
                    outputs = llm.chat(batch_prompts, 
                                    sampling_params=sampling_params, 
                                    use_tqdm=False)

                    # Check correctness for the batch
                    is_correct = [reward_func(each.outputs[0].text, answer) for each, answer in zip(outputs, batch_answers)]
                    # Add detailed results for each item in the batch
                    for j in range(len(batch_prompts)):
                        result_entry = {
                            "prompt": batch_prompts[j],
                            "answer": batch_answers[j],
                            "output": outputs[j].outputs[0].text,  # Assuming this is how to get the output
                            "correctness": is_correct[j]
                        }
                        all_results.append(result_entry)
                except:
                    print(k, i, i+n)
                    # for each in batch_prompts:
                    #     print(each)
                    #correct=[each['correctness'] for each in all_results]
                    #print(f"Correct: {np.mean(np.array(correct).flatten())}")
                    if k == n_repeat-1:
                        return all_results
                break
            
            if i%5 == 0:
                correct=[each['correctness'] for each in all_results]
                pbar.set_postfix(accuracy=np.mean(np.array(correct).flatten()))

    # Save results to JSON file
    #save_results(all_results)
    correct=[each['correctness'] for each in all_results]
    print(f"Correct: {np.mean(np.array(correct).flatten())}")
    return all_results

def save_results(results, filename="data/results/evaluation_tom.json"):
    """Saves the evaluation results to a JSON file."""
    with open(filename, mode='w') as file:
        json.dump(results, file, indent=4)
    print(f"Results saved to {filename}")

**384 prompts model**

Processing batches: 100%|██████████| 381/381 [00:54<00:00,  7.04it/s, accuracy=0.679]

Correct: 0.6792152704135737

**512 prompts model 773 check**

Processing batches: 100%|██████████| 381/381 [00:47<00:00,  7.99it/s, accuracy=0.729]

Correct: 0.7290899526564966

**512 prompts model 300 check**

Processing batches: 100%|██████████| 381/381 [00:51<00:00,  7.38it/s, accuracy=0.706]

Correct: 0.7056962025316456

# Alll Test

In [10]:
test_data_all = load_dataset('parquet', data_files='data/test_data_all.parquet')['train']
test_half_ft = load_dataset('parquet', data_files='data/cleaned_train_ft_with_structure_384.parquet', split='train[50%:]')

In [11]:
test_half_ft

Dataset({
    features: ['qprop=params', 'param=num_people', 'level_2', 'question', 'answer', 'qprop=nth_order', 'qprop=non_unique_mental_state', 'sprop=is_false_belief_story_1st', 'sprop=is_false_belief_story_1st_and_2nd', 'sprop=story_accuracy_1st_raw', 'sprop=story_accuracy_1st_infilled', 'sprop=global_idx', 'param=story_type', 'param=num_stories_total', 'param=max_sentences', 'param=num_moves', 'param=num_rooms', 'data_source', 'prompt', 'ability', 'reward_model', 'extra_info', 'story', 'story_type'],
    num_rows: 2215
})

**384 model**

Processing batches: 100%|██████████| 533/533 [01:21<00:00,  6.54it/s, accuracy=0.632]

Correct: 0.6317606444188723

**512 token 773**

Processing batches: 100%|██████████| 533/533 [01:10<00:00,  7.57it/s, accuracy=0.685]

Correct: 0.6851990984222389

**512 token 300**

Processing batches: 100%|██████████| 533/533 [01:15<00:00,  7.02it/s, accuracy=0.66] 

Correct: 0.6595022624434389

In [12]:
results_all = main_func(llm, test_half_ft, 5, n_repeat=10)

Processing batches:  29%|██▉       | 129/443 [00:30<02:20,  2.23it/s, accuracy=0.661]

0 635 640


Processing batches:  54%|█████▍    | 240/443 [00:57<00:40,  4.96it/s, accuracy=0.65] 

0 1190 1195


Processing batches: 100%|██████████| 443/443 [01:43<00:00,  4.26it/s, accuracy=0.659]

Correct: 0.6594104308390023


In [13]:
dfre = pd.DataFrame(results_all)
dfre.head(2)

,prompt,answer,output,correctness
0,[{'content': 'You are a helpful assistant. The...,does not know about it,<think> reasoning process: Jasmine's comments ...,1
1,[{'content': 'You are a helpful assistant. The...,does not know about it,<think> reasoning process: Jasmine and Emily a...,1


In [14]:
dfre.loc[dfre.answer.isin(['yes', 'no'])].groupby(['answer']).correctness.mean()

answer
no     0.0
yes    1.0
Name: correctness, dtype: float64

In [15]:
results_all = main_func(llm, test_data_all, 5, n_repeat=10)

Processing batches:  18%|█▊        | 94/533 [00:24<01:43,  4.26it/s, accuracy=0.654]

0 465 470


Processing batches:  30%|███       | 160/533 [00:40<02:15,  2.76it/s, accuracy=0.665]

0 795 800


Processing batches: 100%|██████████| 533/533 [02:16<00:00,  3.90it/s, accuracy=0.661]

Correct: 0.6606334841628959


# Other Set

In [17]:
test_faux = load_dataset('parquet', data_files='data/test/test_faux.parquet')['train']
test_hitom = load_dataset('parquet', data_files='data/test/test_hitom.parquet')['train']
test_tomi = load_dataset('parquet', data_files='data/test/test_tomi.parquet')['train']
test_hitom_fixed = load_dataset('parquet', data_files='data/test/test_hitom_fixed.parquet')['train']

In [18]:
test_hitom

Dataset({
    features: ['deception', 'story_length', 'question_order', 'sample_id', 'story', 'question', 'choices', 'answer', 'question_old', 'answer_old', 'prompt'],
    num_rows: 600
})

In [19]:
test_hitom['prompt'][0]

[{'content': 'You are a helpful assistant. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think><answer> answer here </answer>.',
  'role': 'system'},
 {'content': 'Story: Charlotte, Isabella, Elizabeth and Owen entered the living_room. lettuce is in the green_drawer. moved the lettuce to the green_bathtub. dislikes the tangerine. exited the living_room. moved the lettuce to the blue_pantry. likes the green_envelope. exited the living_room. made no movements and stayed in the living_room for 1 minute. exited the living_room. dislikes the green_pepper. made no movements and stayed in the living_room for 1 minute. exited the living_room. moved the lettuce to the green_drawer. exited the living_room. Charlotte, Isabella, Elizabeth and Owen entered the waiting_room.\n Question:

In [20]:
outputs= llm.chat([test_hitom['prompt'][0], test_hitom['prompt'][1], test_hitom['prompt'][2], test_hitom['prompt'][3]], sampling_params)
print(outputs[0].outputs[0].text)
print(test_hitom['answer'][0])

print(outputs[1].outputs[0].text)
print(test_hitom['answer'][1])

print(outputs[2].outputs[0].text)
print(test_hitom['answer'][2])

print(outputs[3].outputs[0].text)
print(test_hitom['answer'][3])

Processed prompts: 100%|██████████| 4/4 [00:00<00:00,  8.69it/s, est. speed input: 1880.21 toks/s, output: 534.71 toks/s]

<think> reasoning process: The question mentions moving the lettuce to the living room, green_drawer, green_bathtub, and blue_pantry. The story states that the lettuce was in the green_drawer, green_bathtub, and blue_pantry before being moved. </think>
<answer> living room </answer>
green_drawer
<think> reasoning process: The story describes Carter, Elizabeth, Hannah, and Jacob entering and exiting the bedroom multiple times, with the tomato being in different containers. We need to determine where the tomato was really. </think>
<answer> bedroom </answer>
blue_container
<think> reasoning process: The eggplant is in the green_envelope because it was initially in the pantry. It then moved to the green_basket, then to the green_bathtub, and finally to the green_envelope. </think>
<answer> green_envelope </answer>
green_envelope
<think> reasoning: Emily, Avery, Evelyn, and Mila are in the front yard, moving the spinach to the green_bucket, then exiting the yard, and returning to the front

In [21]:
results_hitom = main_func(llm, test_hitom, 5, n_repeat=3)

Processing batches:   3%|▎         | 4/120 [00:02<01:35,  1.22it/s, accuracy=0.133]

0 15 20


Processing batches:  13%|█▎        | 16/120 [00:07<01:01,  1.68it/s, accuracy=0.0429]

0 75 80


Processing batches:  60%|██████    | 72/120 [00:25<00:36,  1.33it/s, accuracy=0.0638]

0 355 360


Processing batches: 100%|██████████| 120/120 [00:42<00:00,  2.81it/s, accuracy=0.0821]

Correct: 0.08205128205128205


In [24]:
results_tomi = main_func(llm, test_tomi[:400], 5, n_repeat=3)

Processing batches:  57%|█████▊    | 46/80 [00:12<00:19,  1.74it/s, accuracy=0.28] 

0 225 230


Processing batches:  76%|███████▋  | 61/80 [00:16<00:10,  1.79it/s, accuracy=0.302]

0 300 305


Processing batches: 100%|██████████| 80/80 [00:21<00:00,  3.79it/s, accuracy=0.287]

Correct: 0.28717948717948716


In [25]:
outputs= llm.chat([test_faux['prompt'][0], test_faux['prompt'][1], test_faux['prompt'][2]], sampling_params)
print(outputs[0].outputs[0].text)
print(test_faux['answer'][0])

print(outputs[1].outputs[0].text)
print(test_faux['answer'][1])

print(outputs[2].outputs[0].text)
print(test_faux['answer'][2])

Processed prompts: 100%|██████████| 3/3 [00:00<00:00,  6.41it/s, est. speed input: 1085.18 toks/s, output: 361.71 toks/s]

<think> reasoning process: The story ends with Emma saying "I'm sorry about your story" and Alice immediately saying "What do you mean? " </think> <answer> yes </answer>
Yes
<think> reasoning process: Alice's response to Emma's apology suggests she felt it was inappropriate to express regret. </think> <answer> They should not have said "I'm sorry about your story." </answer>
I'm sorry about your story.
<think> reasoning process: Emma and Alice were both participants in the story competition. Emma wanted to win, but Alice was the winner. Emma was away from school, so she didn't participate in the competition. Alice saw Emma and said she was sorry for her story. Since Alice was the winner, Emma cannot win by saying she was sorry. </think> <answer> Emma </answer>
Alice


In [26]:
results_faux = main_func(llm, test_faux, 5, n_repeat=3)

Processing batches:  58%|█████▊    | 21/36 [00:05<00:03,  4.41it/s, accuracy=0.22] 

0 95 100


Processing batches: 100%|██████████| 36/36 [00:08<00:00,  4.21it/s, accuracy=0.222]

Correct: 0.2222222222222222


In [26]:
model_name = 'NS_512_ckpt773'
pd.DataFrame(results_hitom).to_csv('data/results/hitom_{}.csv'.format(model_name), index=False)
pd.DataFrame(results_tomi).to_csv('data/results/tomi_{}.csv'.format(model_name), index=False)
pd.DataFrame(results_faux).to_csv('data/results/faux_{}.csv'.format(model_name), index=False)

# 0.5B Model 
## WS 384
### HiTom
Processing batches: 100%|██████████| 120/120 [00:26<00:00,  4.51it/s, accuracy=0.25] 

Correct: 0.24957264957264957

### ToMi
Processing batches: 100%|██████████| 1199/1199 [03:20<00:00,  5.97it/s, accuracy=0.409]

Correct: 0.4088702928870293

### Faux Pas
Processing batches: 100%|██████████| 36/36 [00:05<00:00,  7.17it/s, accuracy=0.261]

Correct: 0.2608695652173913

## NS 512 773ckpt
### HiTom
Processing batches: 100%|██████████| 120/120 [00:30<00:00,  3.90it/s, accuracy=0.25] 

Correct: 0.24957264957264957

### ToMi
Processing batches: 100%|██████████| 1199/1199 [03:21<00:00,  5.94it/s, accuracy=0.391]

Correct: 0.3912605042016807

### Faux Pas
Processing batches: 100%|██████████| 36/36 [00:05<00:00,  6.23it/s, accuracy=0.247]

Correct: 0.2469879518072289